# Code for downloading OSM nodes and calculate accessibility measures with pandana and save to json

## Import 

In [47]:
import plotly.express as px
import pandas as pd
import pandana, time, os, pandas as pd, numpy as np
from pandana.loaders import osm
import plotly

## Settings for pandana

In [57]:
amenities = ['restaurant', 'bar', 'school', 'hospital']
distance = 1000
num_pois = 10
num_categories = len(amenities) + 1 #one for each amenity, plus one extra for all of them combined

# Bounding box
bbox = [55.6236813, 12.5202081, 55.6842218, 12.5950132]


# strings for saving files

bbox_string = '_'.join([str(x) for x in bbox])
net_filename = 'data/network_{}.h5'.format(bbox_string)
poi_filename = 'data/pois_{}_{}.csv'.format('_'.join(amenities), bbox_string)

## Download data from OSM and create network

In [ ]:
start_time = time.time()
if os.path.isfile(poi_filename):
    # if a points-of-interest file already exists, just load the dataset from that
    pois = pd.read_csv(poi_filename)
    method = 'loaded from CSV'
else:   
    # otherwise, query the OSM API for the specified amenities within the bounding box 
    osm_tags = '"amenity"~"{}"'.format('|'.join(amenities))
    pois = osm.node_query(lat_min=bbox[0],lng_min=bbox[1], lat_max=bbox[2], lng_max=bbox[3], tags=osm_tags)
    
    # using the '"amenity"~"school"' returns preschools etc, so drop any that aren't just 'school' then save to CSV
    pois = pois[pois['amenity'].isin(amenities)]
    pois.to_csv(poi_filename, index=False, encoding='utf-8')
    method = 'downloaded from OSM'
    
print('{:,} POIs {} in {:,.2f} seconds'.format(len(pois), method, time.time()-start_time))
pois[['amenity', 'name', 'lat', 'lon']].head()


start_time = time.time()
if os.path.isfile(net_filename):
    # if a street network file already exists, just load the dataset from that
    network = pandana.network.Network.from_hdf5(net_filename)
    method = 'loaded from HDF5' 
else:
    #otherwise, query the OSM API for the street network within the specified bounding box
    network = osm.pdna_network_from_bbox(lat_min=bbox[0],lng_min=bbox[1], lat_max=bbox[2], lng_max=bbox[3])
    method = 'downloaded from OSM'

    # identify nodes that are connected to fewer than some threshold of other nodes within a given distance
    lcn = network.low_connectivity_nodes(impedance=1000, count=10, imp_name='distance')
    network.save_hdf5(net_filename, rm_nodes=lcn) #remove low-connectivity nodes and save to h5

print('Network with {:,} nodes {} in {:,.2f} secs'.format(len(network.node_ids), method, time.time()-start_time))
network.precompute(distance + 1)
# initialize the underlying C++ points-of-interest engine
network.init_pois(num_categories=num_categories, max_dist=distance, max_pois=num_pois)
# initialize a category for all amenities with the locations specified by the lon and lat columns
network.set_pois(category='all', x_col=pois['lon'], y_col=pois['lat'])
# initialize each amenity category with the locations specified by the lon and lat columns
for amenity in amenities:
    pois_subset = pois[pois['amenity']==amenity]
    network.set_pois(category=amenity, x_col=pois_subset['lon'], y_col=pois_subset['lat'])



## Create column for distance to nearest amenity for each amenity type

In [59]:


temp_df = network.nodes_df

for ame in amenities:
    temp_df[f'{ame}_distance'] = network.nearest_pois(distance=distance, category=f'{ame}', num_pois=num_pois)[1]


In [60]:
temp_df

,x,y,restaurant_distance,bar_distance,school_distance,hospital_distance
id,,,,,,
118725,12.562906,55.673985,95.097000,130.087997,162.432999,1000.0
118744,12.560034,55.661718,213.123001,639.276001,958.510986,1000.0
118759,12.553866,55.657275,859.354004,550.807007,1000.000000,1000.0
118763,12.542231,55.654003,246.557999,246.557999,1000.000000,1000.0
118764,12.542389,55.653946,258.287994,258.287994,1000.000000,1000.0
...,...,...,...,...,...,...
9156241448,12.569180,55.684281,71.079002,132.781006,342.744995,1000.0
9156241503,12.523896,55.655803,1000.000000,977.495972,1000.000000,1000.0
9156241508,12.523926,55.655679,1000.000000,963.684998,1000.000000,1000.0


In [61]:
temp_df.to_json(path_or_buf="copenhagen_pandana_nodes.json")